In [1]:
import os
import sys
import pickle
import random

import networkx as nx
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import SAGEConv
from torch_geometric.utils import to_undirected
from torch_geometric.utils.convert import from_networkx

# Add "src" path to Python path
sys.path.append(os.path.abspath("../src"))

# Import custom graph formatting function
from graph_formatting_utils import format_graph_for_graphsage
from models import GraphSAGE
from losses import unsupervised_loss
from train import train_in_cpu


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Usuario\pyenvs\python311_graphsage\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\Usuario\pyenvs\python311_graphsage\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Usuario\pyenvs\python311_graphsage\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

In [2]:
# Check CUDA status
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")
print("CUDA Device Count:", torch.cuda.device_count())

CUDA Available: False
GPU Name: No GPU detected
CUDA Device Count: 0


In [3]:
# This should be removed from this notebook
with open("../data/multihop_graph_w_sem_embeddings.pkl", "rb") as f:
    G = pickle.load(f)

cleaned_G = format_graph_for_graphsage(G, embedding_dim=1024)

In [5]:
# Convert the NetworkX graph to a PyTorch Geometric Data object
data = from_networkx(cleaned_G)

# Ensure the graph is undirected
data.edge_index = to_undirected(data.edge_index)

# Create data attribute "x" containing the embeddings of each node complying with the PyTorch Geometric API
data.x = data.embedding

# Instantiate the GraphSAGE model
model = GraphSAGE(
    in_channels=1024,   # Input features (BGE-M3 embeddings)
    hidden_channels=512,  # First hidden layer (alto para máxima capacidad)
    out_channels=256,   # Output embeddings (más ricos)
    num_layers=2        # Mantenemos 2 capas (2 hops)
)

# Set device for model training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model = model.to(device)

# Instantiate the NeighborLoader for mini-batch training
train_loader = NeighborLoader(
    data,
    num_neighbors=[25, 15],  # 25 neighbors for the first layer, 15 for the second
    batch_size=512,  # Batch size
    shuffle=True
)

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Define scaler if GPU is available
scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

In [ ]:
if torch.cuda.is_available():
    train_in_cpu(model, train_loader, optimizer, num_epochs=5, loss_fn=unsupervised_loss, debug=True)

elif torch.cuda.is_available():
    pass # Not yet implemented


In [ ]:
train_in_cpu(model, train_loader, optimizer, num_epochs=5, loss_fn=unsupervised_loss, debug=True)

Training epochs:   0%|          | 0/5 [00:00<?, ?it/s]

[DEBUG] Batch shapes - batch.x: torch.Size([2773, 1024]), batch.edge_index: torch.Size([2, 8382])
[DEBUG] Model output (z) shape: torch.Size([2773, 256])
[DEBUG] First 5 embeddings: tensor([[-0.0101, -0.4896,  0.1358,  ..., -0.7491,  0.0576,  0.1241],
        [ 0.0436, -0.3235, -0.1622,  ..., -0.5725, -0.0701, -0.1153],
        [ 0.1624, -0.2986, -0.0804,  ..., -0.6393, -0.1481, -0.5903],
        [ 0.3485, -0.3122, -0.0451,  ..., -0.5101,  0.2627, -0.1359],
        [ 0.2253,  0.0853, -0.2239,  ..., -0.7649,  0.4128, -0.1385]],
       grad_fn=<SliceBackward0>)
[DEBUG] Loss value for this batch: 25.5706


[DEBUG] Accumulated loss so far: 25.5706
[DEBUG] Batch shapes - batch.x: torch.Size([2798, 1024]), batch.edge_index: torch.Size([2, 8838])
[DEBUG] Model output (z) shape: torch.Size([2798, 256])
[DEBUG] First 5 embeddings: tensor([[ 0.0621, -0.0660, -0.2876,  ..., -0.4607, -0.0407, -0.0907],
        [ 0.2154, -0.0383, -0.1937,  ..., -0.6689,  0.2169, -0.5168],
        [ 0.0945, -0.0185,  0.0920,  ..., -0.6374,  0.5040, -0.1033],
        [ 0.3072, -0.1978,  0.0140,  ..., -0.8093, -0.0475, -0.3139],
        [ 0.1989, -0.3348, -0.1228,  ..., -0.2762, -0.0823,  0.1049]],
       grad_fn=<SliceBackward0>)
[DEBUG] Loss value for this batch: 14.5771


[DEBUG] Accumulated loss so far: 40.1477
[DEBUG] Batch shapes - batch.x: torch.Size([2829, 1024]), batch.edge_index: torch.Size([2, 9014])
[DEBUG] Model output (z) shape: torch.Size([2829, 256])
[DEBUG] First 5 embeddings: tensor([[ 0.4718,  0.0713,  0.0891,  ..., -0.3726, -0.4004, -0.0769],
        [ 0.1594,  0.0526, -0.0022,  ..., -0.5512, -0.0227, -0.2478],
        [ 0.1313, -0.1231,  0.0910,  ..., -0.4452, -0.0829, -0.0795],
        [ 0.1714, -0.0217,  0.1649,  ..., -0.8307, -0.2780, -0.3727],
        [ 0.2161, -0.2287,  0.0990,  ..., -0.4482, -0.1093, -0.0223]],
       grad_fn=<SliceBackward0>)
[DEBUG] Loss value for this batch: 7.8850


[DEBUG] Accumulated loss so far: 48.0328
[DEBUG] Batch shapes - batch.x: torch.Size([2854, 1024]), batch.edge_index: torch.Size([2, 8864])
[DEBUG] Model output (z) shape: torch.Size([2854, 256])
[DEBUG] First 5 embeddings: tensor([[ 0.5466, -0.0970,  0.1330,  ..., -0.4868, -0.5332, -0.0880],
        [ 0.2894, -0.1147,  0.0031,  ..., -0.4096, -0.4167,  0.0618],
        [ 0.1758, -0.1613, -0.0102,  ..., -0.5733, -0.3134, -0.1464],
        [ 0.3503,  0.0716,  0.0202,  ..., -0.6801, -0.2435, -0.5106],
        [-0.1137, -0.1539,  0.0058,  ..., -0.3366, -0.1248, -0.0309]],
       grad_fn=<SliceBackward0>)
[DEBUG] Loss value for this batch: 4.5439


...

[DEBUG] Accumulated loss so far: 3.5817
[DEBUG] Batch shapes - batch.x: torch.Size([2930, 1024]), batch.edge_index: torch.Size([2, 8998])
[DEBUG] Model output (z) shape: torch.Size([2930, 256])
[DEBUG] First 5 embeddings: tensor([[ 0.4096,  0.0749,  0.4121,  ..., -0.6657, -0.3841, -0.7085],
        [ 0.3127,  0.1978,  0.3142,  ..., -0.8632, -0.4962, -0.8640],
        [ 0.6495,  0.2107,  0.1181,  ..., -0.1865, -0.2582, -0.5027],
        [ 0.5478,  0.2318,  0.3735,  ..., -0.6707, -0.6845, -0.5810],
        [ 0.4532,  0.0515,  0.4222,  ..., -0.3597, -0.4222, -0.6993]],
       grad_fn=<SliceBackward0>)
[DEBUG] Loss value for this batch: 1.7217


[DEBUG] Accumulated loss so far: 5.3034
[DEBUG] Batch shapes - batch.x: torch.Size([2797, 1024]), batch.edge_index: torch.Size([2, 8619])
[DEBUG] Model output (z) shape: torch.Size([2797, 256])
[DEBUG] First 5 embeddings: tensor([[ 0.3510, -0.1994,  0.2283,  ..., -0.4365, -0.8389, -0.4603],
        [ 0.3388,  0.0074,  0.4425,  ..., -0.1792, -0.5969, -0.5290],
        [ 0.4168,  0.0289,  0.3785,  ..., -0.3978, -0.4384, -0.4695],
        [ 0.3633, -0.0554,  0.1131,  ..., -0.3959, -0.6639, -0.4465],
        [ 0.4129,  0.0451,  0.6237,  ..., -0.2535, -0.2577, -0.4858]],
       grad_fn=<SliceBackward0>)
[DEBUG] Loss value for this batch: 1.6373


[DEBUG] Accumulated loss so far: 6.9407
[DEBUG] Batch shapes - batch.x: torch.Size([2716, 1024]), batch.edge_index: torch.Size([2, 8338])
[DEBUG] Model output (z) shape: torch.Size([2716, 256])
[DEBUG] First 5 embeddings: tensor([[ 0.5027,  0.0290,  0.5306,  ..., -0.5743, -0.3836, -0.8134],
        [ 0.3461, -0.0043,  0.5708,  ..., -0.7560, -0.2998, -0.2275],
        [ 0.4561,  0.0297,  0.5090,  ..., -0.4550, -0.0548, -0.4251],
        [ 0.3477, -0.0216,  0.2632,  ..., -0.4864, -0.5904, -0.3621],
        [ 0.5156,  0.0961,  0.4605,  ..., -0.4799, -0.5555, -0.4197]],
       grad_fn=<SliceBackward0>)
[DEBUG] Loss value for this batch: 1.6771


[DEBUG] Accumulated loss so far: 8.6178
[DEBUG] Batch shapes - batch.x: torch.Size([2872, 1024]), batch.edge_index: torch.Size([2, 9101])
[DEBUG] Model output (z) shape: torch.Size([2872, 256])
[DEBUG] First 5 embeddings: tensor([[ 0.4203, -0.2545,  0.4813,  ..., -0.7020, -0.6237, -0.3962],
        [ 0.3759, -0.2042,  0.1762,  ..., -0.3605, -0.4915, -0.5231],
        [ 0.4724, -0.0147,  0.2795,  ..., -0.4084, -0.5542, -0.5146],
        [ 0.4535, -0.1075,  0.4356,  ..., -0.5183, -0.6925, -0.4122],
        [ 0.6190, -0.1299,  0.4555,  ..., -0.6974, -0.2556, -0.5016]],
       grad_fn=<SliceBackward0>)
[DEBUG] Loss value for this batch: 1.5323


[DEBUG] Accumulated loss so far: 10.1500
[DEBUG] Batch shapes - batch.x: torch.Size([2444, 1024]), batch.edge_index: torch.Size([2, 6446])
[DEBUG] Model output (z) shape: torch.Size([2444, 256])
[DEBUG] First 5 embeddings: tensor([[ 0.2809,  0.0512,  0.5101,  ..., -0.6815, -0.4657, -0.6115],
        [ 0.4843, -0.0774,  0.4034,  ..., -0.6615, -0.3689, -0.7927],
        [ 0.2194,  0.0791,  0.3112,  ..., -0.3385, -0.4928, -0.3674],
        [ 0.7366, -0.0839,  0.3669,  ..., -0.7824, -0.4686, -0.7601],
        [ 0.2341,  0.1427,  0.5700,  ..., -0.4824, -0.4742, -0.4194]],
       grad_fn=<SliceBackward0>)
[DEBUG] Loss value for this batch: 2.2345


In [16]:
# Save embeddings as .npy file

## 1) Get the device from any parameter in the model
#device = next(model.parameters()).device
#
## 2) Now move your data.x and data.edge_index to that device:
#data_x = data.x.to(device)
#data_edge_index = data.edge_index.to(device)
#
## 3) Forward pass with torch.no_grad():
#with torch.no_grad():
#    embeddings = model(data_x, data_edge_index)
#
#embeddings_np = embeddings.cpu().numpy()
#np.save("../data/graphsage_embeddings.npy", embeddings_np)
#print("Embeddings saved to graphsage_embeddings.npy")


In [17]:
## Add Graph_SAGE embeddings to the baseline graph
#
## 1) Move data to the same device as the model
#device = next(model.parameters()).device
#data_x = data.x.to(device)
#data_edge_index = data.edge_index.to(device)
#
## 2) Obtain final embeddings from the trained model
#with torch.no_grad():
#    final_emb = model(data_x, data_edge_index)  # shape [num_nodes, embedding_dim]
#    final_emb_np = final_emb.cpu().numpy()
#
## 3) Add them back to the cleaned_G graph
#list_of_nodes = list(G.nodes())  # Must match the node ordering in data
#for i, node in enumerate(list_of_nodes):
#    # Store as a NumPy array (or you could store as a list if you prefer)
#    G.nodes[node]["SAGE_embedding"] = final_emb_np[i]
#
#print("SAGE embeddings added to G under 'SAGE_embedding' attribute.")
#

SAGE embeddings added to G under 'SAGE_embedding' attribute.


In [18]:
#with open(f"../data/multihop_graph_w_sage{num_epochs}_embeddings_1hop.pkl", "wb") as f:
#    pickle.dump(G, f)

In [19]:
## Save the model's state_dict to disk
#torch.save(model.state_dict(), f"../data/graphsage_{num_epochs}_model_1hop.pth")
#print("Model saved to graphsage_model.pth")
#

Model saved to graphsage_model.pth


In [20]:
#def get_new_node_embedding(model, new_feature, device, self_loop=True):
#    """
#    Generate an embedding for a new node using the trained GraphSAGE model.
#    
#    Parameters:
#      model (torch.nn.Module): The trained GraphSAGE model.
#      new_feature (torch.Tensor): The new node's initial features of shape (1024,).
#      device (torch.device): The device (e.g., cuda) on which the model is located.
#      self_loop (bool, default=True): If True, adds a self-loop edge (node connected to itself).
#                                      This simulates neighbor aggregation when no neighbors exist.
#    
#    Returns:
#      torch.Tensor: The new node's embedding of shape (out_channels,).
#    """
#    # Set the model to evaluation mode.
#    model.eval()
#    
#    # Move the new node's features to the correct device.
#    new_feature = new_feature.to(device)
#    
#    if self_loop:
#        # Create a self-loop edge_index. This indicates that the node is connected to itself.
#        # The edge_index tensor must have shape [2, num_edges]; here we create a single edge (0,0).
#        edge_index = torch.tensor([[0], [0]], dtype=torch.long, device=device)
#    else:
#        # Alternatively, if you prefer no edges, you can pass an empty edge_index.
#        # Note: Without a self-loop, the model may not transform the features as intended.
#        edge_index = torch.empty((2, 0), dtype=torch.long, device=device)
#    
#    # GraphSAGE expects a batch dimension, so unsqueeze new_feature to shape [1, 1024].
#    with torch.no_grad():
#        new_embedding = model(new_feature.unsqueeze(0), edge_index)
#    
#    # Remove the batch dimension to return a tensor of shape [out_channels].
#    return new_embedding.squeeze(0)
#
## Example usage:
## Assuming 'model' is your trained GraphSAGE and you have a new node feature vector of size 1024.
#new_feature = torch.randn(1024)  # Replace with the actual feature vector for the new node.
#device = next(model.parameters()).device  # Get the device from the model.
#new_node_embedding = get_new_node_embedding(model, new_feature, device, self_loop=True)
#print("New node embedding shape:", new_node_embedding.shape)
#

New node embedding shape: torch.Size([256])
